<a href = "https://www.pieriantraining.com"><img src="../PT Centered Purple.png"> </a>

<em style="text-align:center">Copyrighted by Pierian Training</em>

## Reddit API:
1. Go to https://www.reddit.com/prefs/apps
2. Click "create application"
3. Name: My Sentiment Example
4. Choose script
5. Leave *description* and *about url* blank
6. Add http://www.example.com/unused/redirect/uri to *redirect uri*

## Imports

In [1]:
# instlaa reddit api package
#!pip install praw

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 191.0/191.0 kB 755.7 kB/s eta 0:00:00a 0:00:01


In [2]:
import praw
import openai
import os

In [4]:
# os.environ["REDDIT_CLIENT_ID"] = ""
# os.environ["REDDIT_CLIENT_SECRET"] = ""


In [6]:
reddit_client_id = os.getenv("REDDIT_CLIENT_ID")
reddit_client_secret = os.getenv("REDDIT_CLIENT_SECRET")

In [7]:
reddit = praw.Reddit(client_id=reddit_client_id, # the App ID retreived from the reddit application
                     client_secret=reddit_client_secret, # the secret retreived from the reddit application
                     user_agent="sentiment analysis test by u/Sdet-Devops"  # add your reddit name
                    )

We can now already use our reddit instance to obtain public information from reddit!

In [8]:
for submission in reddit.subreddit("test").hot(limit=5):
    print(submission.title)


test
app
test
post1
post1
post3
post2
post1
Test Image Upload
test


Of course we want to access hot subreddits like r/wallstreetbets or r/stocks

In [9]:
subreddit_stocks = reddit.subreddit("stocks")

In [5]:
print(subreddit_stocks.display_name)
print(subreddit_stocks.title)
print(subreddit_stocks.accounts_active)

stocks
Stocks - Investing and trading for all
5977


Let's say we want to access the first 5 hot posts and three most upvoted comments:

In [10]:
for post in subreddit_stocks.hot(limit=5):
    print(post.title)
    submission = reddit.submission(post.id)
    c = 0
    # print top 2 comments per title comment (5 titles in all)
    for comment in submission.comments:
        comment = comment.body
        if not comment == "[deleted]": # ignore deleted comment
            print(comment)
            c+=1
        if c == 2:
            break

Rate My Portfolio - r/Stocks Quarterly Thread June 2023
ABBV- AbbVie Inc

APD- Air Products & Chemicals, Inc.

GOOGL- Alphabet Inc Class A

AMZN - Amazon.com, Inc.

AWK - American Water Works Company Inc

ADSK - Autodesk Inc

BRK.B - Berkshire Hathaway Inc Class B

CHTR - Charter Communications Inc

COST - Costco Wholesale Corporation

CCI -Crown Castle Inc

DG - Dollar General Corp

HSY - Hershey Co

INTC - Intel Corporation

IP - International Paper Co

LOW - Lowe's Companies Inc

MA - Mastercard Inc

MSFT - Microsoft Corp

PGR - Progressive Corp

PLD - Prologis Inc

PSA - Public Storage

VALE - Vale SA

WBA - Walgreens Boots Alliance Inc

WM - Waste Management, Inc.
Let me know what you think about my portfolio. Trying to stay away from the sectors I don't know and increase diversity by investing VOO and QQQ ETFs  


17.39% AAPL

13.86% GOOGL

26.31% VOO

12.63% BAC

04.99% TQQQ

11.47% AMZN

13.37% QQQM
/r/Stocks Weekend Discussion Saturday - Aug 05, 2023
Is anyone aware of any UK/

We can now use GPT to extract stock tickers from all of these posts and tell us whether the sentiment for the mentioned assets is positive, neutral or negative:

## OpenAI API

Remember to use the notebook as shown, you must set your OpenAI API Key as an environment variable. Obviously, there are many ways you could provide your API Key to the Python code, input() or even hard-coded, but those are typically not recommended for safety reasons. Having it as an environment variable let's the key live on the computer, but not actually be present in the code.

### Set-up Open AI API Key

We'll only need to do this once per computer

In [12]:
# Uncomment below and swap in your key to place your environment key using Python
# Then you can delete the key string and the code cell below will still work!
# os.environ["OPENAI_API_KEY"] = "Your key goes here!

In [13]:
openai.api_key = os.getenv("OPENAI_API_KEY")

## Prompt Generation
First, we need to create a dictionary containing the title and top three comments

In [14]:
getattr(subreddit_stocks, "hot")

<bound method BaseListingMixin.hot of Subreddit(display_name='stocks')>

In [15]:
def get_titles_and_comments(subreddit="stocks", sub_instance="hot", limit=10, num_comments=2, skip_first=2):
    subreddit = reddit.subreddit(subreddit)
    titles_and_comments = {}
    for c, post in enumerate(getattr(subreddit, sub_instance)(limit=limit)):
        # c is counter
        if c < skip_first:
            continue
        
        c+=(1-skip_first)
        
        titles_and_comments[c] = ""

        submission = reddit.submission(post.id)
        title = post.title
        
        titles_and_comments[c] += "Title: " + title  + "\n\n"
        titles_and_comments[c] += "Comments: \n\n"
        
        comment_counter = 0
        for comment in submission.comments:
            comment = comment.body
            if not comment == "[deleted]":
                titles_and_comments[c] += comment + "\n"
                comment_counter+=1
            if comment_counter == num_comments:
                break

    return titles_and_comments

In [16]:
titles_and_comments = get_titles_and_comments(subreddit="stocks", limit=12)

In [17]:
print(titles_and_comments[2])

Title: Fed’s Bowman says more U.S. rate hikes likely will be needed

Comments: 

Color me shocked
There will be atleast one more rate hike of 25 basis points. Will there be one more? thats the question. 

Just because the Fed stops rate hikes doesn't mean its going to start cutting them. I think this time they might just hold them for a bit till inflation is where they want it to be.



In [18]:
def create_prompt(title_and_comments):

    task = "Return the stock ticker or company in the following heading and comments and classify the sentiment. If no ticker or company is mentioned write 'No company mentioned':\n\n"
    return task + title_and_comments


In [19]:
prompt = create_prompt(titles_and_comments[1])

In [21]:
print(prompt)

Return the stock ticker or company in the following heading and comments and classify the sentiment. If no ticker or company is mentioned write 'No company mentioned':

Title: Musk found not liable in securities class action suit

Comments: 

This investor has suffered so much harm. Stock up only 800% since that fraud tweet. 

So much irreparable damage.
No dog in this fight outside of owning tesla shares indirectly from index funds, but what would stop a company, say Jones Soda (JSDA) from saying Saudi Arabia wants to take them private at a 30% premium. 

Does this jury verdict reset what the norm is for corporate finance- from an adjudicating stand point? Being able to tweet/email/DM that you're being purchased to increase market cap with no repercussions?



In [26]:
for key, title_and_comments in titles_and_comments.items():
    
    prompt = create_prompt(title_and_comments)
    
    response = openai.Completion.create(engine="text-davinci-003",
                                                prompt=prompt,
                                                max_tokens=256,
                                                temperature=0,
                                                top_p=1.0,
                                                frequency_penalty=0.0,
                                                presence_penalty=0.0)
    print(title_and_comments)
    print("Sentiment: " + response["choices"][0]["text"])
    print("-"*30)

Title: Musk found not liable in securities class action suit

Comments: 

This investor has suffered so much harm. Stock up only 800% since that fraud tweet. 

So much irreparable damage.
No dog in this fight outside of owning tesla shares indirectly from index funds, but what would stop a company, say Jones Soda (JSDA) from saying Saudi Arabia wants to take them private at a 30% premium. 

Does this jury verdict reset what the norm is for corporate finance- from an adjudicating stand point? Being able to tweet/email/DM that you're being purchased to increase market cap with no repercussions?

Sentiment: 
No company mentioned
------------------------------
Title: December jobs blowout: +517k added (vs +187k expected); unemployment rate hits 3.4%

Comments: 

Insane... Even in a high rates environment

I wonder how the fed responds in medium term
Holy balls lol

Sentiment: 
No company mentioned
------------------------------
Title: Amazon, Meta, Alphabet, and Microsoft will incur $10B+ 